# SANSANのコンタクトのCSVをSQLServerに登録する

In [ ]:
!pip install pyodbc
!pip install sqlalchemy==1.4.46
!pip install pandas

In [1]:
import pandas as pd
from sqlalchemy import create_engine

# ユーザ(SANSAN_USER)のTRUNCATE＋登録

In [6]:
dtype = {
    'ユーザID': 'Int64',
    # 'ＦＩＬ': 'Int64',
    '削除フラグ': 'Int64',
}

csv_file = r"data\sansan_user.csv"
user_df = pd.read_csv(csv_file, dtype=dtype)
column_mappings = {
    '所属部署 *': 'dpt',
    'ユーザID *': 'id',
    'ユーザ名 *': 'name',
    # 'ユーザ名の読み': 'NAME_READABLE',
    'メールアドレス *': 'mail',
    # 'サブメールアドレス': 'SUB_MAIL',
    # '利用開始日 *': 'START_DATE',
    # '言語 (システム上の表示言語)': 'LANG',
    # 'メール受信形式': 'MAILER',
    # 'SAML Name ID': 'SAML_NAME_ID',
    # 'ユーザ区分': 'CATERGORY',
    # '[権限] 全データ更新': 'AUTH_UPDATE',
    # '[権限] 名刺・コンタクトのダウンロード': 'AUTH_DL_CARD',
    # '[権限] メール配信': 'AUTH_MAIL',
    # '[権限] 案件管理': 'AUTH_ANKEN',
    # '[権限] API・Zapier連携': 'AUTH_API',
    # '[権限] 利用実績の確認': 'AUTH_RECORD',
    # '[権限] 会社・組織ツリーのダウンロード': 'AUTH_DL_CO',
    # 'アカウント状況': 'STATUS',
    '削除フラグ': 'del_flg',
}

req_columns = list(column_mappings.keys())
fltr_user_df = user_df[req_columns]
fltr_user_df = fltr_user_df.rename(columns=column_mappings)

# 部署に'産業資材事業部'を含む行のみを選択
pattern = '産業資材事業部'
fltr_user_df = fltr_user_df[fltr_user_df['dpt'].str.contains(pattern, regex=True)]

# 最初のスペースまでの文字列を抽出
fltr_user_df['branch'] = fltr_user_df['dpt'].str.split().str[0]

fltr_user_df['del_flg'].fillna(0, inplace=True)

print(fltr_user_df)


               dpt               id   name                        mail  \
3       本社　産業資材事業部         t-yoshik  吉川 高央         t-yoshik@risu.co.jp   
4       本社　産業資材事業部         y-otsuka  大塚 泰尚         y-otsuka@risu.co.jp   
5       本社　産業資材事業部  sanshi-tenjikai  産資展示会  sanshi-tenjikai@risu.co.jp   
6       本社　産業資材事業部         k-miyajm  宮嶋 一泰         k-miyajm@risu.co.jp   
7       本社　産業資材事業部          n-kondo  近藤 伸彦          n-kondo@risu.co.jp   
..             ...              ...    ...                         ...   
81    福岡支店　産業資材事業部        r-kaijima   貝嶋 亮        r-kaijima@risu.co.jp   
82    福岡支店　産業資材事業部        taka-itou  伊藤 崇洋        taka-itou@risu.co.jp   
83    福岡支店　産業資材事業部        t-hayashi   林 大雅        t-hayashi@risu.co.jp   
84  鹿児島営業所　産業資材事業部         y-hayasi   林 義博         y-hayasi@risu.co.jp   
85  鹿児島営業所　産業資材事業部          t-nisio  西尾 孝之          t-nisio@risu.co.jp   

    del_flg  branch  
3         0      本社  
4         0      本社  
5         0      本社  
6         0      本社  
7

In [7]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text

# SQL Serverへの接続設定
server = 'GPKMSQ14'
database = 'datamart'
username = 'gpkadmin'
password = '19vK8xEQ'
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'

# SQLAlchemyエンジンを作成
engine = create_engine(connection_string)

In [ ]:
# Truncateするテーブル名
user_table_name = "sansan_user"

# テーブルをtruncateする
with engine.begin() as conn:
    conn.execute(text(f"TRUNCATE TABLE {user_table_name}"))


In [ ]:
# SQL Serverにデータをインポート
fltr_user_df.to_sql(user_table_name, con=engine, if_exists='append', index=False)

# コンタクト登録

In [11]:
csv_file = r"data\contactlist_ 20230321_20231126_utf8.csv"
contact_df = pd.read_csv(csv_file)
contact_df = contact_df.drop(['社外出席者 [部署 役職]'], axis=1)
column_mappings = {
    'コンタクト日': 'contact_date',
    '時間': 'contact_time',
    '区分': 'catergory',
    '重要フラグ': 'importance',
    '訪問目的': 'purpose',
    '訪問先': 'destination',
    '商談アイテム': 'product',
    '社外出席者': 'customer',
    '部署': 'customer_dept',
    '役職': 'customor_pos',
    '社内出席者': 'salesperson_visited',
    'メモ': 'contact_details',
    '登録者': 'salesperson_registered',
}
req_columns = list(column_mappings.keys())
fltr_contact_df = contact_df[req_columns]

fltr_contact_df.rename(columns=column_mappings, inplace=True)

fltr_contact_df['importance'] = fltr_contact_df['importance'].fillna('')

# 最初のスペースまでの文字列を抽出
fltr_contact_df['customer_company'] = fltr_contact_df['customer'].str.split().str[0]

# 2つ目のスペースまでの文字列を抽出
fltr_contact_df['customer_name'] = fltr_contact_df['customer'].apply(lambda x: ' '.join(x.split()[1:3]))


# ユーザを名前で結合
user_brn_df = fltr_user_df[list(['name', 'branch'])]
merged_df = fltr_contact_df.merge(user_brn_df, left_on='salesperson_registered', right_on='name' , how='inner').drop(columns=['name'])
merged_df = merged_df.fillna('')

print(merged_df)

     contact_date contact_time catergory importance         purpose  \
0      2023/11/26  13:00～15:30    面会（来訪）             2.フォロー訪問（計画訪問）   
1      2023/11/22  16:00～17:00    面会（訪問）             2.フォロー訪問（計画訪問）   
2      2023/11/21  16:00～18:00    面会（来訪）             2.フォロー訪問（計画訪問）   
3      2023/11/20  17:30～18:00    電話（発信）             2.フォロー訪問（計画訪問）   
4      2023/11/14  10:00～12:00    面会（訪問）             2.フォロー訪問（計画訪問）   
...           ...          ...       ...        ...             ...   
9484   2023/05/25  14:00～15:30    面会（訪問）                   1.新規顧客訪問   
9485   2023/04/18  10:00～11:30        面会                   1.新規顧客訪問   
9486   2023/04/17  16:00～17:00    面会（訪問）                      3.その他   
9487   2023/03/23  17:00～18:00    面会（来訪）             2.フォロー訪問（計画訪問）   
9488   2023/03/22  14:30～15:30    面会（訪問）             2.フォロー訪問（計画訪問）   

     destination    product                                         customer  \
0         ２．ユーザー  ４．ＴＥＣＣＥＬＬ  株式会社クボタ 菊池 俊介\r\n株式会社落合 樋口 智一\r\n株式会社カ

In [12]:
# SQL Serverへの接続設定
server = 'GPKMSQ14'
database = 'datamart'
username = 'gpkadmin'
password = '19vK8xEQ'
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'

# SQLAlchemyエンジンを作成
engine = create_engine(connection_string)

# SQL Serverにデータをインポート
# 1万件で13分かかる
merged_df.to_sql('sansan_contact', con=engine, if_exists='append', index=False)

57